# 🧼 Retail Sales Data Cleaning and Preparation (Portfolio Project 1)

This is my first end-to-end portfolio project where I demonstrate the complete data cleaning and preparation workflow using **Python (Pandas)** and exporting the cleaned dataset into a **MySQL database**, which will later be connected to **Power BI** for building reports and dashboards.

---

## 📌 Project Overview

**Source Data:** Online Retail Dataset (Excel file)\
**Tools Used:** Python, Pandas, MySQL, SQLAlchemy, Power BI\
**Goal:** Prepare clean, structured data ready for analysis and visualization

---

## 📁 Step 1: Import Required Libraries

```python


In [6]:
import pandas as pd


## 📥 Step 2: Load the Excel Data into a DataFrame

In [3]:
df = pd.read_excel("Data/Online Retail.xlsx")


In [14]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [16]:
df.shape

(541909, 8)

In [18]:
df.dtypes

InvoiceNo              object
StockCode              object
Description            object
Quantity                int64
InvoiceDate    datetime64[ns]
UnitPrice             float64
CustomerID            float64
Country                object
dtype: object

In [20]:
df.describe().round(2).T

,count,mean,min,25%,50%,75%,max,std
Quantity,541909.0,9.55,-80995.0,1.0,3.0,10.0,80995.0,218.08
InvoiceDate,541909,2011-07-04 13:34:57.156386048,2010-12-01 08:26:00,2011-03-28 11:34:00,2011-07-19 17:17:00,2011-10-19 11:27:00,2011-12-09 12:50:00,NaN
UnitPrice,541909.0,4.61,-11062.06,1.25,2.08,4.13,38970.0,96.76
CustomerID,406829.0,15287.69,12346.0,13953.0,15152.0,16791.0,18287.0,1713.6


## ❓ Step 3: Check for Missing Values

In [24]:
print(df.isnull().sum())

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64


### Calculate % of missing values


In [29]:
missing_values_percentage = df.isnull().mean() * 100
missing_filtered = missing_values_percentage[missing_values_percentage > 0]
missing_with_symbol = missing_filtered.apply(lambda x: f"{x: .2f} %")
print(missing_with_symbol)

Description      0.27 %
CustomerID      24.93 %
dtype: object


## 🧹 Step 4: Clean the Data

### Drop rows with missing product descriptions


In [34]:
df = df.dropna(subset=['Description'])

### Drop rows with missing customer IDs

In [38]:
df = df.dropna(subset=['CustomerID'])

## 🔄 Step 5: Tag Sales vs Returns

### Check number of returns

In [49]:
sales_qty = df[df['Quantity'] > 0].shape[0]
returns_qty = df.shape[0] - sales_qty
print(f"Total Sales Quantity: \033[1m{sales_qty:,}\033[0m")

Total Sales Quantity: 397,924


In [51]:
print(f"Total Return Quantity: \033[1m{returns_qty:,}\033[0m")

Total Return Quantity: 8,905


### Tag transaction type

In [54]:
df['Transaction_Type'] = df['Quantity'].apply(lambda x: 'Return' if x < 0 else 'Sale')

## 💰 Step 6: Create Revenue Column

In [61]:
df['TotalPrice'] = df['Quantity'] * df['UnitPrice']

## 📅 Step 7: Parse Dates and Extract Components


In [65]:
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
df['InvoiceYear'] = df['InvoiceDate'].dt.year
df['InvoiceQuarter'] = df['InvoiceDate'].dt.quarter
df['InvoiceMonth'] = df['InvoiceDate'].dt.month

## 🧾 Step 8: Remove Duplicate Rows

In [75]:
duplicate_rows = df[df.duplicated()]
print(f"Number of Duplicate rows: \033[1m{duplicate_rows.shape[0]:,}\033[0m")

Number of Duplicate rows: 5,225


### Keep only the first occurrence

In [78]:
df = df.drop_duplicates()

## 📝 Data Cleaning Summary
1. Dropped missing values from Description and CustomerID for valid product and customer-level insights

2. Tagged each transaction as a Sale or Return based on Quantity

3. Created TotalPrice for each line item to compute revenue

4. Converted InvoiceDate to datetime and extracted Year, Quarter, and Month

5. Removed duplicates to maintain data consistency


## 💾 Step 9: Export Cleaned Data to CSV

In [90]:
df.to_csv('Data/cleaned_online_retail.csv', index=False)

## 🛢️ Step 10: Export Cleaned Data to MySQL

### Install Required Libraries

In [96]:
!pip install mysql-connector-python
!pip install sqlalchemy

### Create SQLAlchemy Engine and Export Data

In [99]:
from sqlalchemy import create_engine
import urllib.parse

In [123]:
host = "localhost"
user = "root"
password = urllib.parse.quote_plus("your_mysql_password")  # encode special characters
database = "portfolio1_retail_sales_analysis"

In [119]:
connection = create_engine(f"mysql+mysqlconnector://{user}:{password}@{host}/{database}")

In [121]:
df.to_sql(name='retail_sales_cleaned', con=connection, if_exists='replace', index=False)

401604

## 🔄 Next Steps
1. Create fact and dimension tables from the cleaned data in MySQL
2. Use Power BI to connect to MySQL and build dashboards
3. Perform RFM Analysis, Sales Trends, Customer Segmentation, and more!

## 📌 Tags
Python Pandas MySQL SQLAlchemy ETL Data Cleaning Portfolio Project Power BI

⭐ Thanks for checking out my project! Stay tuned for the Power BI report and insights coming soon.